# Load Data

In [4]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_v2')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_v2


In [5]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
underage = np.zeros(6) #underage data
overage = np.zeros(6) #overage data

for i in range(6):
    p = prices[i]
    c = costs[i]
    s = salvages[i]
    underage[i] = p - c
    overage[i] = c - s


alpha = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])

num_simulations = 1000  # Number of simulations for estimating expected profit

# Preprocessing

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# apply one hot encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_train['location'])

onehot_encoder = OneHotEncoder()
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded).toarray()  # Convert to dense array

# Replace the original column with the one-hot encoded data
data_train = data_train.drop('location', axis=1)
data_train = pd.concat([data_train, pd.DataFrame(onehot_encoded)], axis=1)

# apply one hot encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_test['location'])

onehot_encoder = OneHotEncoder()
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded).toarray()  # Convert to dense array

# Replace the original column with the one-hot encoded data
data_test = data_test.drop('location', axis=1)
data_test = pd.concat([data_test, pd.DataFrame(onehot_encoded)], axis=1)


   day  month  year  is_holiday  temperature    0    1    2    3    4    5  \
0    8      7  2021       False         9.37  0.0  0.0  0.0  0.0  0.0  1.0   
1    2     12  2020       False         9.96  0.0  1.0  0.0  0.0  0.0  0.0   
2   15      5  2021       False         9.37  0.0  0.0  0.0  0.0  0.0  1.0   
3   13      2  2020       False         9.37  0.0  0.0  0.0  0.0  0.0  1.0   
4    9      6  2022       False        11.68  0.0  0.0  1.0  0.0  0.0  0.0   

     6    7    8    9  
0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  


# ANN

In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from scipy.stats import reciprocal


# Custom loss function adapted for vectorized critical ratios and substitution matrix
def newsvendor_quantile_loss_with_substitution(u, o, y_true, y_pred, alpha):

    # Cast numpy arrays to tensors
    u = tf.convert_to_tensor(u, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(o, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha, dtype=tf.float32) #substitution matrix
    
    # Calculate unmet demand for each product
    unmet_demand = tf.maximum(0.0, y_pred - y_true)

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = tf.zeros(6, dtype=tf.float32)
    for i in range(6):
        demand_increase += unmet_demand[i] * alpha[i]
    #demand_increase = tf.tensordot(alpha, unmet_demand, axes=[[0], [0]])

    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    # Compute the loss with adjusted demand
    loss = -tf.reduce_mean(u * y_pred - (u + o) * tf.maximum(y_pred - adjusted_demand, 0))
    return loss


# Model creation function 
def create_model(n_hidden, n_neurons, learning_rate, activation, underage, overage, alpha, input_shape=[15]):
    model = Sequential([Dense(n_neurons, activation=activation, input_shape=input_shape)] +
                       [Dense(n_neurons, activation=activation) for _ in range(n_hidden)] +
                       [Dense(6)])  # 6 output variables
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss=lambda y_true, y_pred: newsvendor_quantile_loss_with_substitution(underage, overage, y_true, y_pred, alpha))
    return model




# Model builder function 
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3, activation='relu', underage=np.ones(6), overage=np.ones(6), alpha=np.eye(6)):
    return KerasRegressor(model=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, 
                          learning_rate=learning_rate, activation=activation, 
                          underage=underage, overage=overage, alpha=alpha)

# Create a model
model_ANN = model_builder()

# Define the parameter grid
from scipy.stats import reciprocal
param_distribs = {
    "n_hidden": range(0, 11),
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Optimize the model using RandomizedSearchCV
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=20, cv=5, scoring='neg_mean_squared_error')

# Fit the model
target_train = target_train.astype('float32')
rnd_search_cv_ANN.fit(data_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Make predictions
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)
target_pred_ANN = pd.DataFrame(np.row_stack(target_pred_ANN))

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_ANN)
print("MSE:", mse)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in

{'activation': 'tanh', 'batch_size': 32, 'epochs': 30, 'learning_rate': 0.0014244450000789198, 'n_hidden': 10, 'n_neurons': 62}
MSE: 374.67596638227474


# DT

In [14]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'estimator__boosting_type': ['gbdt', 'dart', 'goss'],
    'estimator__num_leaves': [10, 20, 30, 40, 50],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__feature_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_freq': [3, 4, 5, 6, 7],
}

critical_ratio = np.mean(underage) / (np.mean(overage) + np.mean(underage))
cr = critical_ratio

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=cr, metric='quantile', verbose=0)

# Wrap the model with MultiOutputRegressor
model = MultiOutputRegressor(model)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(data_train, target_train)
print(rnd_search.best_params_)

# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(data_test)
target_pred_DT = pd.DataFrame(np.row_stack(target_pred_DT))
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\la

# Costs

In [15]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0

for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0

        if target_pred_ANN.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_ANN.iloc[i, j]))

        if target_pred_ANN.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN = (costs[j] - salvages[j]) * (np.round(target_pred_ANN.iloc[i, j]) - target_test.iloc[i, j])
        
        if target_pred_DT.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_DT.iloc[i, j]))

        if target_pred_DT.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT = (costs[j] - salvages[j]) * (np.round(target_pred_DT.iloc[i, j]) - target_test.iloc[i, j])
        
        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT

# Print the overall costs
print('Overall costs for ANN: ', int(overall_costs_ANN))
print('Overall costs for DT: ', int(overall_costs_DT))

Overall costs for ANN:  1672
Overall costs for DT:  329
